# Flask: WebApp using sqlite3

Prerequistes:
*    pip install flask
*    pip install flask_sqlalchemy

In [316]:
cd '/home/chris/workspace/python-course/Level 4/02 Flask'
ls -l
rm -rf workspace2
ls -l

total 76
-rw-rw-r-- 1 chris chris  5853 Dec  5 13:45 03.ipynb
-rw-rw-r-- 1 chris chris  8359 Dec  4 22:18 1.basic_webapp.ipynb
-rw-rw-r-- 1 chris chris 18851 Dec  6 16:38 2.database_webapp.ipynb
-rw-rw-r-- 1 chris chris   901 Dec  6 13:25 app.py
-rw-rw-r-- 1 chris chris   901 Dec  6 14:48 database_app.py
-rw-r--r-- 1 chris chris 12288 Dec  6 14:48 database.db
drwxrwxr-x 2 chris chris  4096 Dec  6 14:48 __pycache__
-rw-rw-r-- 1 chris chris    72 Dec  5 13:35 Untitled1.ipynb
-rw-rw-r-- 1 chris chris    72 Dec  4 12:17 Untitled.ipynb
drwxrwxr-x 2 chris chris  4096 Dec  6 16:39 workspace2
total 72
-rw-rw-r-- 1 chris chris  5853 Dec  5 13:45 03.ipynb
-rw-rw-r-- 1 chris chris  8359 Dec  4 22:18 1.basic_webapp.ipynb
-rw-rw-r-- 1 chris chris 18851 Dec  6 16:38 2.database_webapp.ipynb
-rw-rw-r-- 1 chris chris   901 Dec  6 13:25 app.py
-rw-rw-r-- 1 chris chris   901 Dec  6 14:48 database_app.py
-rw-r--r-- 1 chris chris 12288 Dec  6 14:48 database.db
drwxrwxr-x 2 chris chris  4096 Dec  6 14:48 __

In [317]:
stty -echo
mkdir workspace2
cd workspace2

Create a basic webapp

In [318]:
# create new app
cat << EOF > database_app.py
import os
from flask import Flask, render_template, request, url_for, redirect
from flask_sqlalchemy import SQLAlchemy

from sqlalchemy.sql import func


basedir = os.path.abspath(os.path.dirname(__file__))

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] =\
        'sqlite:///' + os.path.join(basedir, 'database.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)


class Student(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    firstname = db.Column(db.String(100), nullable=False)
    lastname = db.Column(db.String(100), nullable=False)
    email = db.Column(db.String(80), unique=True, nullable=False)
    age = db.Column(db.Integer)
    created_at = db.Column(db.DateTime(timezone=True),
                           server_default=func.now())
    bio = db.Column(db.Text)

    def __repr__(self):
        return f'<Student {self.firstname}>'

@app.route('/')
def index():
    students = Student.query.all()
    return render_template('index.html', students=students)

EOF

Start the Flask development server:

In [319]:
fuser -k 8000/tcp  # kill previous incarnations
flask --app database_app run --host localhost --port 8000 &

[1] 52712


In [320]:
export FLASK_APP=database_app
flask shell << EOF
from database_app import db, Student
db.create_all()
db.drop_all()
db.create_all()
EOF

 * Serving Flask app 'database_app'
 * Debug mode: off
 * Running on http://localhost:8000
Press CTRL+C to quit
Python 3.10.15 (main, Oct 18 2024, 15:04:49) [GCC 13.2.0] on linux
App: database_app
Instance: /home/chris/workspace/python-course/src/53 Flask/workspace2/instance
>>> >>> >>> >>> >>> 
now exiting InteractiveConsole...


In [321]:
export FLASK_APP=database_app
flask shell << EOF
john = Student(firstname='john', lastname='doe',
                       email='jd@example.com', age=23,
                       bio='Biology student')
sammy = Student(firstname='Sammy',
               lastname='Shark',
               email='sammyshark@example.com',
               age=20,
               bio='Marine biology student')

carl = Student(firstname='Carl',
               lastname='White',
               email='carlwhite@example.com',
               age=22,
               bio='Marine geology student')

db.session.add(john)
db.session.add(sammy)
db.session.add(carl)
db.session.commit()
Student.query.all()
EOF

Python 3.10.15 (main, Oct 18 2024, 15:04:49) [GCC 13.2.0] on linux
App: database_app
Instance: /home/chris/workspace/python-course/src/53 Flask/workspace2/instance
>>> ... ... >>> ... ... ... ... >>> >>> ... ... ... ... >>> >>> >>> >>> >>> >>> [<Student john>, <Student Sammy>, <Student Carl>]
>>> 
now exiting InteractiveConsole...


Create the template.  This needs to be stored in the `template` subdirectory.  
Note how we link to the stylesheet:  
```    <link rel="stylesheet" type= "text/css" href= "{{ url_for('static',filename='styles/mystyles.css') }}">```

The above expands to:  
```    <link rel="stylesheet" type= "text/css" href= "/static/styles/mystyles.css">```

In [322]:
mkdir templates
cat << EOF > templates/base.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <link rel="stylesheet" type= "text/css" href= "{{ url_for('static',filename='styles/mystyles.css') }}"    
    <title>{% block title %} {% endblock %} - FlaskApp</title>
</head>
<body>
    <nav>
        <a href="{{ url_for('index') }}">FlaskApp</a>
        <a href="#">Create</a>
        <a href="#">About</a>
    </nav>
    <hr>
    <div class="content">
        {% block content %} {% endblock %}
    </div>
</body>
</html>
EOF

In [323]:
mkdir -p static/styles
cat << EOF > static/styles/mystyles.css
.title {
    margin: 5px;
}

.content {
    margin: 5px;
    width: 100%;
    display: flex;
    flex-direction: row;
    flex-wrap: wrap;
}

.student {
    flex: 20%;
    padding: 10px;
    margin: 5px;
    background-color: #f3f3f3;
    inline-size: 100%;
}

.bio {
    padding: 10px;
    margin: 5px;
    background-color: #ffffff;
    color: #004835;
}

.name a {
    color: #00a36f;
    text-decoration: none;
}

nav a {
    color: #d64161;
    font-size: 3em;
    margin-left: 50px;
    text-decoration: none;
}
EOF

In [324]:
cat << EOF > templates/index.html-
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title> - FlaskApp</title>
    <link rel="stylesheet" type= "text/css" href= "{{ url_for('static',filename='styles/mystyles.css') }}"    
</head>
<body>
    <nav>
        <a href="{{ url_for('index') }}">FlaskApp</a>
        <a href="#">Create</a>
        <a href="#">About</a>
    </nav>
    <hr>
    <h1 class="title">Students</h1>
    <div class="content">
        {% for student in students %}
            <div class="student">
                <p><b>#{{ student.id }}</b></p>
                <b>
                    <p class="name">{{ student.firstname }} {{ student.lastname }}</p>
                </b>
                <p>{{ student.email }}</p>
                <p>{{ student.age }} years old.</p>
                <p>Joined: {{ student.created_at }}</p>
                <div class="bio">
                    <h4>Bio</h4>
                    <p>{{ student.bio }}</p>
                </div>
            </div>
        {% endfor %}
    </div>
</body>
</html>
EOF

In [325]:
cat << EOF > templates/index.html
{% extends 'base.html' %}

{% block content %}
    <h1 class="title">{% block title %} Students {% endblock %}</h1>
    <div class="content">
        {% for student in students %}
            <div class="student">
                <p><b>#{{ student.id }}</b></p>
                <b>
                    <p class="name">{{ student.firstname }} {{ student.lastname }}</p>
                </b>
                <p>{{ student.email }}</p>
                <p>{{ student.age }} years old.</p>
                <p>Joined: {{ student.created_at }}</p>
                <div class="bio">
                    <h4>Bio</h4>
                    <p>{{ student.bio }}</p>
                </div>
            </div>
        {% endfor %}
    </div>
{% endblock %}
EOF

In [326]:
#export FLASK_APP=database_app
#export FLASK_ENV=development
#flask run

In [327]:
fuser -k 8000/tcp  # kill previous incarnations
flask --app database_app run --host localhost --port 8000 &

8000/tcp:            52712
[2] 52894
[1]   Killed                  flask --app database_app run --host localhost --port 8000


Now display views in firefox

In [328]:
firefox http://localhost:8000
#firefox --new-tab http://localhost:8000/view1
#firefox --new-tab http://localhost:8000/view2
#firefox --new-tab http://localhost:8000/week/7
#firefox --new-tab http://localhost:8000/convert/23.9

 * Serving Flask app 'database_app'
 * Debug mode: off
 * Running on http://localhost:8000
Press CTRL+C to quit


In [329]:
flask shell << EOF
from database_app import db, Student
Student.query.filter_by(firstname='Sammy').all()
Student.query.filter_by(firstname='Sammy').first()
Student.query.get(3)
exit()
EOF

127.0.0.1 - - [06/Dec/2024 16:39:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2024 16:39:53] "GET /static/styles/mystyles.css HTTP/1.1" 200 -
Python 3.10.15 (main, Oct 18 2024, 15:04:49) [GCC 13.2.0] on linux
App: database_app
Instance: /home/chris/workspace/python-course/src/53 Flask/workspace2/instance
>>> >>> [<Student Sammy>]
>>> <Student Sammy>
>>> <Student Carl>
>>> 


In [330]:
cat << EOF >> database_app.py
# ...

@app.route('/<int:student_id>/')
def student(student_id):
    student = Student.query.get_or_404(student_id)
    return render_template('student.html', student=student)
EOF

In [331]:
cat << EOF > templates/student.html
{% extends 'base.html' %}

{% block content %}
    <span class="title">
        <h1>{% block title %} {{ student.firstname }} {{ student.lastname }}{% endblock %}</h1>
    </span>
    <div class="content">
            <div class="student">
                <p><b>#{{ student.id }}</b></p>
                <b>
                    <p class="name">{{ student.firstname }} {{ student.lastname }}</p>
                </b>
                <p>{{ student.email }}</p>
                <p>{{ student.age }} years old.</p>
                <p>Joined: {{ student.created_at }}</p>
                <div class="bio">
                    <h4>Bio</h4>
                    <p>{{ student.bio }}</p>
                </div>
            </div>
    </div>
{% endblock %}
EOF

In [333]:
fuser -k 8000/tcp  # kill previous incarnations
flask --app database_app run --host localhost --port 8000 &

127.0.0.1 - - [06/Dec/2024 16:40:12] "GET /2 HTTP/1.1" 404 -
8000/tcp:            52894
[3] 53437
[2]   Killed                  flask --app database_app run --host localhost --port 8000


In [334]:
firefox http://localhost:8000/2
#firefox --new-tab http://localhost:8000/view1
#firefox --new-tab http://localhost:8000/view2
#firefox --new-tab http://localhost:8000/week/7
#firefox --new-tab http://localhost:8000/convert/23.9

 * Serving Flask app 'database_app'
 * Debug mode: off
 * Running on http://localhost:8000
Press CTRL+C to quit


In [335]:
cat << EOF > templates/index.html
{% extends 'base.html' %}

{% block content %}
    <h1 class="title">{% block title %} Students {% endblock %}</h1>
    <div class="content">
{% for student in students %}
    <div class="student">
        <p><b>#{{ student.id }}</b></p>
        <b>
            <p class="name">
                <a href="{{ url_for('student', student_id=student.id)}}">
                    {{ student.firstname }} {{ student.lastname }}
                </a>
            </p>
        </b>
        <p>{{ student.email }}</p>
        <p>{{ student.age }} years old.</p>
        <p>Joined: {{ student.created_at }}</p>
        <div class="bio">
            <h4>Bio</h4>
            <p>{{ student.bio }}</p>
        </div>
    </div>
{% endfor %}    </div>
{% endblock %}
EOF

127.0.0.1 - - [06/Dec/2024 16:40:24] "GET /2 HTTP/1.1" 308 -
127.0.0.1 - - [06/Dec/2024 16:40:24] "GET /2/ HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2024 16:40:24] "GET /static/styles/mystyles.css HTTP/1.1" 304 -


In [336]:
fuser -k 8000/tcp  # kill previous incarnations
flask --app database_app run --host localhost --port 8000 &

8000/tcp:            53437
[4] 54842
[3]   Killed                  flask --app database_app run --host localhost --port 8000


In [337]:
firefox http://localhost:8000/2
#firefox --new-tab http://localhost:8000/view1
#firefox --new-tab http://localhost:8000/view2
#firefox --new-tab http://localhost:8000/week/7
#firefox --new-tab http://localhost:8000/convert/23.9

 * Serving Flask app 'database_app'
 * Debug mode: off
 * Running on http://localhost:8000
Press CTRL+C to quit


In [338]:
cat << EOF >> database_app.py

@app.route('/create/', methods=('GET', 'POST'))
def create():
    return render_template('create.html')
EOF

127.0.0.1 - - [06/Dec/2024 17:04:47] "GET /2/ HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2024 17:04:47] "GET /static/styles/mystyles.css HTTP/1.1" 304 -


In [344]:
cat << EOF > templates/create.html
{% extends 'base.html' %}

{% block content %}
    <h1 style="width: 100%">{% block title %} Add a New Student {% endblock %}</h1>
    <form method="post">
        <p>
            <label for="firstname">First Name</label>
            <input type="text" name="firstname"
                   placeholder="First name">
            </input>
        </p>

        <p>
            <label for="lastname">Last Name</label>
            <input type="text" name="lastname"
                   placeholder="Last name">
            </input>
        </p>

        <p>
            <label for="email">Email</label>
            <input type="email" name="email"
                   placeholder="Student email">
            </input>
        </p>

        <p>
            <label for="age">Age</label>
            <input type="number" name="age"
                   placeholder="Age">
            </input>
        </p>

        <p>
        <label for="bio">Bio</label>
        <br>
        <textarea name="bio"
                  placeholder="Bio"
                  rows="15"
                  cols="60"
                  ></textarea>
        </p>
        <p>
            <button type="submit">Submit</button>
        </p>
    </form>
{% endblock %}
EOF


[2024-12-06 17:09:10,051] ERROR in app: Exception on /create/ [GET]
Traceback (most recent call last):
  File "/home/chris/.pyenv/versions/3.10.15/lib/python3.10/site-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/chris/.pyenv/versions/3.10.15/lib/python3.10/site-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/chris/.pyenv/versions/3.10.15/lib/python3.10/site-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/chris/.pyenv/versions/3.10.15/lib/python3.10/site-packages/flask/app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "/home/chris/workspace/python-course/src/53 Flask/workspace2/database_app.py", line 44, in create
    return render_template('create.html')
  File "/home/chris/.pyenv/versions/3.10.15/lib/pyt

In [345]:
fuser -k 8000/tcp  # kill previous incarnations
flask --app database_app run --host localhost --port 8000 &

8000/tcp:            55569
[6] 56032
[5]   Killed                  flask --app database_app run --host localhost --port 8000


In [346]:
firefox http://localhost:8000/create

 * Serving Flask app 'database_app'
 * Debug mode: off
 * Running on http://localhost:8000
Press CTRL+C to quit
